In [ ]:
from __future__ import annotations

from pathlib import Path

In [ ]:
# Read the input file into an iterator
disk_map = Path("day09_input.txt").read_text().strip()

# Part 1

Represent the data as follows:

- `blocks`: A list of file block ids, without any spacing. The number of blocks for each
  file is given by the size of that file.
- `free_space`: A dictionary with key = position and value = space available at that position.

When compacting the data, we will just rearrange the blocks in the list and update the
`free_space` dictionary.


In [ ]:
# Find the blocks and free space on the disk
def disk_layout_part1(disk_map: str) -> tuple[list[int], dict[int, int]]:
    """Get the disk layout from the disk map."""
    blocks, free_space = [], {}
    pos, file_id = 0, 0
    for char_num, char in enumerate(disk_map):
        size = int(char)
        if char_num % 2 == 0:
            blocks.extend([file_id] * size)
            file_id += 1
        else:
            free_space[pos] = size
        pos += size

    return blocks, free_space

In [ ]:
# Compact the disk
def compact_part1(blocks: list[int], free_space: dict[int, int]) -> list[int]:
    """Compact the disk."""
    for pos, num in free_space.items():
        if num == 0:
            continue
        # print(f"Moving {num} blocks to {pos}")
        tail = blocks[-num:]
        tail.reverse()
        # Delete the chunk from the end
        del blocks[-num:]
        # Insert the reversed chunk at the position
        blocks[pos:pos] = tail

    return blocks

In [ ]:
def checksum_part1(blocks: list[int]) -> int:
    """Calculate the checksum of a disk map."""
    return sum(file_id * pos for pos, file_id in enumerate(blocks))

In [ ]:
blocks, free_space = disk_layout_part1(disk_map)
compacted_blocks = compact_part1(blocks, free_space)
checksum_part1(compacted_blocks)

# Part 2

Making new data structures for part 2: Since we're moving whole files: Lets make objects
for each file, and for each segment of free space. When moving a file, we can just
update its starting position and reduce the amount of free space at that location.


In [ ]:
from pydantic import BaseModel


class Space(BaseModel):
    """Free space on the disk."""

    start_pos: int
    size: int

    def __lt__(self, other: Space) -> bool:
        """Sort free space by start position."""
        return self.start_pos < other.start_pos


class File(Space):
    """A file on the disk."""

    id: int


# Find the blocks and free space on the disk
def disk_layout_part2(disk_map: str) -> tuple[list[File], list[Space]]:
    """Get the disk layout from the disk map."""
    spaces, files = [], []
    pos, file_id = 0, 0
    for char_num, char in enumerate(disk_map):
        size = int(char)
        if char_num % 2 == 0:
            files.append(File(id=file_id, start_pos=pos, size=size))
            file_id += 1
        else:
            spaces.append(Space(start_pos=pos, size=size))
        pos += size

    return files, spaces

In [ ]:
def compact_part2(files: list[File], spaces: list[Space]):
    """Compact the disk, Part 2."""
    for file in sorted(files, key=lambda f: f.id, reverse=True):
        for space in sorted(spaces):
            if space.start_pos > file.start_pos:
                # No free space to the left of file
                break
            if space.size >= file.size:
                # print(f"Moving file {file.id} to {space.start_pos}")
                file.start_pos = space.start_pos
                space.start_pos += file.size
                space.size -= file.size
                # Done moving this file
                break

In [ ]:
files, spaces = disk_layout_part2(disk_map)
print(f"There is a total of {len(files):,} files and {len(spaces):,} spaces.")
compact_part2(files, spaces)

In [ ]:
checksum = 0
for file in files:
    for i in range(file.size):
        checksum += file.id * (file.start_pos + i)

checksum